<a href="https://colab.research.google.com/github/vcwild/oula-score/blob/main/oula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# ! wget https://analyse.kmi.open.ac.uk/open_dataset/download --no-check-certificate && mv download download.zip && unzip download.zip && rm download.zip

Archive:  download.zip
  inflating: studentAssessment.csv   
  inflating: studentVle.csv          
  inflating: assessments.csv         
  inflating: courses.csv             
  inflating: studentInfo.csv         
  inflating: studentRegistration.csv  
  inflating: vle.csv                 


In [2]:
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
student_assessment = pd.read_csv('studentAssessment.csv')
student_info = pd.read_csv('studentInfo.csv')
student_registration = pd.read_csv('studentRegistration.csv')
student_vle = pd.read_csv('studentVle.csv')
vle = pd.read_csv('vle.csv')

In [3]:
start_date = abs(min(student_vle.date))
student_grouped = student_vle.groupby(['id_student', 'date']).sum().drop(columns='id_site')
student_reset = student_grouped.reset_index()
student_reset['date_fix'] = [x + start_date for x in student_reset.date]
student_reset['week'] = (student_reset['date_fix'] / 7) + 1
student_reset['week'] = student_reset['week'].apply(np.floor)
student_sum_week = student_reset.groupby(['id_student', 'week']).sum().drop(columns=['date', 'date_fix']).reset_index()

In [4]:
assessments_merge = pd.merge(assessments, student_assessment, how='left', left_on=('id_assessment'), right_on=('id_assessment'))
courses_assessments = pd.merge(courses, assessments_merge, how='left', left_on=('code_module', 'code_presentation'), right_on=('code_module', 'code_presentation'))

In [5]:
courses_assessments['date_fix'] = [x + start_date for x in courses_assessments.date]
courses_assessments['week'] = (courses_assessments['date_fix'] / 7) + 1
courses_assessments['week'] = courses_assessments['week'].apply(np.floor)

In [6]:
df_aux = pd.merge(student_sum_week, courses_assessments, how='left', left_on=('id_student', 'week'), right_on=('id_student', 'week'))
df_aux['click_cumsum'] = df_aux.groupby('id_student').sum_click.cumsum()
df_grouped = df_aux.groupby(['id_student', 'id_assessment']).sum()
df_grouped['click_cumsum_1'] = df_grouped.click_cumsum.shift(1)
df_grouped['diff'] = df_grouped['click_cumsum'] - df_grouped['click_cumsum_1']
df_grouped['diff'] = df_grouped['diff'].clip(lower=0).replace(0, np.nan)
df_grouped['click_cumsum_1'] = df_grouped['click_cumsum_1'].replace(0, np.nan)
df_grouped['mean'] = df_grouped[['click_cumsum', 'click_cumsum_1']].mean(axis=1, skipna=True)
df = df_grouped.reset_index()
df.head()

,id_student,id_assessment,week,sum_click,module_presentation_length,date,weight,date_submitted,is_banked,score,date_fix,click_cumsum,click_cumsum_1,diff,mean
0,6516,1758.0,7.0,101,269.0,19.0,10.0,17.0,0.0,60.0,44.0,653,NaN,NaN,653.0
1,6516,1759.0,12.0,31,269.0,54.0,20.0,51.0,0.0,48.0,79.0,1011,653.0,358.0,832.0
2,6516,1760.0,21.0,69,269.0,117.0,20.0,116.0,0.0,63.0,142.0,1435,1011.0,424.0,1223.0
3,6516,1761.0,28.0,98,269.0,166.0,20.0,164.0,0.0,61.0,191.0,1956,1435.0,521.0,1695.5
4,6516,1762.0,35.0,49,269.0,215.0,30.0,210.0,0.0,77.0,240.0,2494,1956.0,538.0,2225.0


In [16]:
df.corr(method='pearson')['score'].drop(index='score').sort_values(ascending=False)

click_cumsum                  0.142459
mean                          0.133197
click_cumsum_1                0.112654
diff                          0.097088
week                          0.072987
date_fix                      0.072741
date                          0.072741
id_assessment                 0.062106
sum_click                     0.051034
id_student                    0.024027
module_presentation_length    0.017289
is_banked                    -0.011192
date_submitted               -0.015053
weight                       -0.150011
Name: score, dtype: float64

In [17]:
df.corr(method='spearman')['score'].drop(index='score').sort_values(ascending=False)

click_cumsum                  0.157100
mean                          0.143781
click_cumsum_1                0.130427
date_fix                      0.069887
date                          0.069887
diff                          0.068520
week                          0.067891
sum_click                     0.056537
id_student                    0.052174
id_assessment                 0.049779
module_presentation_length    0.025877
is_banked                    -0.016452
date_submitted               -0.022736
weight                       -0.179645
Name: score, dtype: float64

# Preprocessing

In [7]:
df_prep = df.replace(np.nan, 0)

# ML

## Baseline model

In [8]:
features = ['id_student', 'click_cumsum', 'mean', 'click_cumsum_1', 'weight', 'is_banked', 'diff', 'week']

X = df_prep[features]
y = df_prep.score

In [9]:
X_prep = pd.get_dummies(X, columns=['id_student'])

## Train-test split

In [10]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X_prep, y, train_size=0.5, random_state=42)

In [ ]:
# from sklearn.feature_selection import SelectFromModel
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.ensemble import RandomForestRegressor

# # Feature Selection
# k_vs_score=[]
# seed = 42

# for k in range(2, X_train.shape[1], 2):
#     # selector_model = LinearRegression(normalize=True)
#     # selector_model = Ridge(alpha=0.1, normalize=True, random_state=seed)
#     # selector_model = Lasso(alpha=1.,normalize=True, random_state=seed)
#     selector_model = RandomForestRegressor(criterion='mse', random_state=seed, n_jobs=-1)

#     selector = SelectFromModel(selector_model, max_features=k, threshold=-np.inf)
    
#     selector.fit(X_train, y_train)
#     X_train_2 = selector.transform(X_train)

#     X_test_2 = selector.transform(X_test)

#     model = RandomForestRegressor(criterion='mse', n_estimators=100, random_state=seed, n_jobs=-1, max_depth=9)
#     model.fit(X_train_2, y_train)

#     p = model.predict(X_test_2)

#     score = mean_squared_error(y_test, p, squared=False)
#     print(f"k = {k}; MSE = {score}")

#     mask = selector.get_support()
#     print(X_train.columns[mask])
#     k_vs_score.append(score)

In [ ]:
# pd.Series(
#   k_vs_score, 
#   index = range(initial_features, max_n_features, step)).plot(figsize=(6,4), 
#   xticks=range(initial_features, max_n_features, step),
# )
# plt.xlabel('features')
# plt.ylabel('MSE')
# plt.title('perda por MSE');

In [11]:
y_train_valid = y_train[y_train != 0]
X_train_valid =  X_train[y_train != 0]

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(random_state=0, criterion='mse', max_depth=9)

dt.fit(X_train_valid, y_train_valid)

In [ ]:
results = dt.predict(X_test)

In [ ]:
y_metrics = pd.DataFrame()
y_metrics['predict'] = results
y_metrics['test'] = y_test
y_metrics['test'] = y_metrics.test.replace(np.nan, 0)
y_metrics['diff'] = y_metrics.predict - y_metrics.test

In [ ]:
y_metrics.test

In [ ]:
y_metrics['diff'].abs().mean()

In [ ]:
y_metrics

In [ ]:
# valores únicos
df.nunique()

In [ ]:
def score_comparison(by):
  grouped = df.groupby(f'{by}').mean()['score']
  print(grouped, '\n')
  sns.barplot(x=grouped.index, y=grouped.values, palette='Blues')
  plt.xticks(rotation=45)
  plt.title(f'Comparação das notas para {by}')
  plt.show()

In [ ]:
score_comparison('gender')

In [ ]:
score_comparison('highest_education')

In [ ]:
score_comparison('age_band')

In [ ]:
score_comparison('region')

In [ ]:
score_comparison('imd_band')

In [ ]:
score_comparison('sum_click')